# Import


In [2]:
import pandas as pd # for data manipulation
import ast # for safely evaluating string expressions

# Data cleaning

## Perfume dataset exploratory analysis and cleaning

In [5]:
# Load perfume dataset from Excel file into a DataFrame 
df_perfume_catalog = pd.read_excel("Datasets/perfumes_database.xlsx")

In [6]:
# Display dataset dimensions (rows, columns) to understand its overall size
print(df_perfume_catalog.shape)

# Output column names to inspect the structure and available features
print(df_perfume_catalog.columns)

# Preview a random sample of 10 rows to get a quick sense of the data
df_perfume_catalog.sample(10)


(37926, 8)
Index(['brand', 'perfume', 'image', 'launch_year', 'main_accords', 'notes',
       'longevity', 'sillage'],
      dtype='object')


,brand,perfume,image,launch_year,main_accords,notes,longevity,sillage
12697,Creed,Royal Ceylan,o.3807.jpg,2006.0,"[""citrus"", ""green"", ""woody"", ""aromatic"", ""fres...","{""middle"": [""Sea Salt"", ""Tea"", ""Lotus""], ""base...","[""0"", ""0"", ""0"", ""2"", ""1""]","[""1"", ""1"", ""0"", ""4""]"
30721,Paco Rabanne,Lady Million Merry Millions,o.32985.jpg,2015.0,"[""white floral"", ""sweet"", ""honey"", ""citrus"", ""...","{""middle"": [""Jasmine"", ""Orange Blossom"", ""Gard...","[""0"", ""0"", ""1"", ""0"", ""0""]","[""0"", ""1"", ""0"", ""0""]"
35630,Shiseido,Zen Summer,o.5680.jpg,2009.0,"[""citrus"", ""fruity"", ""floral"", ""sweet"", ""woody""]","{""middle"": [""Freesia"", ""Lotus"", ""Gardenia"", ""R...","[""1"", ""0"", ""2"", ""0"", ""0""]","[""0"", ""3"", ""1"", ""0""]"
6226,Leitmotiv,No 51 Etude en Bois,o.38693.jpg,2016.0,"[""whiskey"", ""woody"", ""warm spicy"", ""fresh spic...","[""Coriander"", ""Whiskey"", ""Clove"", ""Jasmine"", ""...","[""0"", ""0"", ""1"", ""2"", ""0""]","[""0"", ""2"", ""1"", ""0""]"
4869,Gap,Stay,o.8738.jpg,2010.0,"[""floral"", ""fresh"", ""woody"", ""white floral"", ""...","{""middle"": [""Lotus"", ""Peony"", ""Bellflower""], ""...","[""3"", ""3"", ""9"", ""0"", ""0""]","[""7"", ""7"", ""4"", ""1""]"
35478,Masaki Matsushima,Aqua Mat Homme,o.3669.jpg,2008.0,"[""citrus"", ""woody"", ""green"", ""fresh"", ""floral""]","{""middle"": [""Magnolia"", ""Bamboo"", ""Brazilian R...","[""1"", ""2"", ""12"", ""6"", ""0""]","[""7"", ""15"", ""4"", ""1""]"
6620,Mariah Carey,Luscious Pink Deluxe Edition Parfum,o.5455.jpg,2009.0,"[""floral"", ""woody"", ""fruity"", ""citrus"", ""fresh...","{""middle"": [""Tiare Flower"", ""Lily-of-the-Valle...","[""1"", ""0"", ""2"", ""9"", ""1""]","[""5"", ""7"", ""7"", ""6""]"
1424,Avon,Far Away Paradise,o.1581.jpg,2007.0,"[""floral"", ""amber"", ""fresh"", ""fruity"", ""vanilla""]","[""Peony"", ""Amber"", ""Vanilla"", ""Peach""]","[""3"", ""0"", ""6"", ""7"", ""3""]","[""3"", ""9"", ""7"", ""4""]"
14363,Guerlain,Aroma Allegoria Aromaparfum Vitalising,o.5846.jpg,2002.0,"[""citrus"", ""green"", ""fresh spicy"", ""fresh"", ""w...","{""middle"": [""Tea"", ""Jasmine"", ""Peach""], ""base""...","[""1"", ""0"", ""0"", ""0"", ""0""]","[""1"", ""2"", ""3"", ""1""]"
17558,Marc de la Morandiere,Thiara,o.9044.jpg,NaN,"[""white floral"", ""tuberose"", ""citrus"", ""warm s...","{""middle"": [""Jasmine"", ""Rose"", ""Cloves"", ""Oran...","[""0"", ""0"", ""0"", ""0"", ""0""]","[""0"", ""0"", ""0"", ""0""]"


In [7]:
# Drop non-informative or irrelevant columns for the analysis to reduce data noise
df_perfume_catalog = df_perfume_catalog.drop(columns = ['image', 'longevity', 'sillage','notes'])

In [8]:
# Inspect data types, duplicates and null value counts to assess data quality and identify potential preprocessing needs
print(df_perfume_catalog.info())
print('Duplicates:', df_perfume_catalog.duplicated().value_counts())
df_perfume_catalog.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37926 entries, 0 to 37925
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         37926 non-null  object 
 1   perfume       37923 non-null  object 
 2   launch_year   26716 non-null  float64
 3   main_accords  36957 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.2+ MB
None
Duplicates: False    37895
True        31
Name: count, dtype: int64


brand               0
perfume             3
launch_year     11210
main_accords      969
dtype: int64

In [9]:
# Identify rows with missing perfume names 
df_perfume_catalog[df_perfume_catalog['perfume'].isna()]

,brand,perfume,launch_year,main_accords
3148,D S Durga,NaN,NaN,"[""musky"", ""tuberose"", ""balsamic"", ""white flora..."
5077,Haught Parfums,NaN,2016.0,"[""white floral"", ""balsamic"", ""powdery"", ""flora..."
33838,Faberlic,NaN,NaN,"[""powdery"", ""floral"", ""sweet"", ""fresh spicy"", ..."


In [10]:
# Define manual corrections for missing perfume names
name_corrections = {
    5077: "&",
    3148: "&",
    33838: "MAMA"
}

# Apply corrections using .loc
for idx, name in name_corrections.items():
    df_perfume_catalog.loc[idx, "perfume"] = name


In [11]:
# Standardize brand and perfume names by converting to lowercase and removing leading/trailing spaces
df_perfume_catalog.brand = df_perfume_catalog.brand.str.lower().str.strip()
df_perfume_catalog.perfume = df_perfume_catalog.perfume.str.lower().str.strip()

In [12]:
# Fill missing launch year values with -1 and convert the column to integer type.
# This ensures consistent data type and allows for easy identification of missing data.
df_perfume_catalog.launch_year = df_perfume_catalog.launch_year.fillna("-1").astype(int)

In [13]:
# Calculate and display the number of missing values in each column of the perfume catalog DataFrame.
df_perfume_catalog.isna().sum()

brand             0
perfume           0
launch_year       0
main_accords    969
dtype: int64

In [14]:
# Replace all missing values in the perfume catalog DataFrame with the string "Unknown".
df_perfume_catalog = df_perfume_catalog.fillna("Unknown")

In [15]:
# Identify the unique data types present in the 'main_accords' column.
# This helps in understanding the data structure and planning appropriate data transformations (e.g., converting to a list).
df_perfume_catalog.main_accords.apply(type).unique()

array([<class 'str'>], dtype=object)

In [16]:
# Convert 'main_accords' column to object type to handle mixed data types.
df_perfume_catalog.main_accords = df_perfume_catalog.main_accords.astype(object)

# Iterate through each entry in the 'main_accords' column and convert strings to lists using ast.literal_eval.
# If an entry contains "Unknown", it is treated as a single-element list.
for index, accords in df_perfume_catalog.main_accords.items():
    if "Unknown" not in accords:
        try:
            accords_list = ast.literal_eval(accords)
        except (SyntaxError, ValueError):
            accords_list = [accords]  # Handle cases where literal_eval fails
    else:
        accords_list = [accords]
    df_perfume_catalog.at[index, "main_accords"] = accords_list

In [17]:
# Display a random sample of 10 rows from the perfume catalog DataFrame 
df_perfume_catalog.sample(10)

,brand,perfume,launch_year,main_accords
25053,rubini,fundamental,2015,"[woody, sweet, powdery, citrus, floral, fruity]"
5206,hollister,skyler,-1,"[fruity, aquatic, floral, sweet, ozonic]"
4554,est e lauder,modern muse nuit,2016,"[balsamic, aromatic, vanilla, powdery, warm sp..."
30350,custo barcelona,custo barcelona,2008,"[citrus, fresh, floral, patchouli, fruity]"
15475,jean paul gaultier,ma dame rose n roll,2009,"[woody, rose, citrus, musky, fresh spicy, fruity]"
29648,parfum bar,napoli mod 1,-1,"[floral, fresh spicy, green, aquatic, ozonic, ..."
22628,fiorucci,i love london,-1,"[white floral, musky, tuberose, citrus, animalic]"
35342,expand,love peace cross,2010,"[citrus, fruity, sweet, floral, musky, woody]"
31566,al battash classic,precious stone amethyst,2015,"[woody, powdery, white floral, rose, vanilla, ..."
36995,natura,water bath estoraque,2008,"[fresh spicy, woody, aromatic, green, balsamic..."


## Extracting and Structuring Fragrance Notes

In [19]:
# Initialize a list to store unique fragrance notes.
unique_fragrance_notes = []

# Iterate through each perfume's main accords to extract and standardize unique notes.
# Each note is converted to lowercase, and "unknown" notes are excluded.
for accord_list in df_perfume_catalog.main_accords: 
    for note in accord_list: 
        note = note.lower() 
        if note != "unknown" and note not in unique_fragrance_notes:
            unique_fragrance_notes.append(note)

# Print the list of all unique fragrance notes and their total count.
print(unique_fragrance_notes, f"\nTotal number of unique notes: {len(unique_fragrance_notes)}")

['wine', 'vanilla', 'sweet', 'woody', 'aromatic', 'musky', 'balsamic', 'powdery', 'warm spicy', 'amber', 'yellow floral', 'white floral', 'oud', 'fresh spicy', 'green', 'tuberose', 'animalic', 'fruity', 'earthy', 'citrus', 'narcotic', 'soapy', 'patchouli', 'herbal', 'rose', 'floral', 'ozonic', 'aquatic', 'cacao', 'fresh', 'aldehydic', 'tropical', 'leather', 'smoky', 'coca-cola', 'gourmand', 'honey', 'sour', 'marine', 'terpenic', 'soft spicy', 'tobacco', 'coconut', 'salty', 'cinnamon', 'rum', 'caramel', 'milky', 'watery', 'nutty', 'bitter', 'almond', 'savory', 'whiskey', 'metallic', 'beeswax', 'sand', 'coffee', 'camphor', 'clay', 'lactonic', 'mossy', 'cherry', 'conifer', 'alcohol', 'vinil', 'vodka', 'asphault', 'caramela', 'industrial glue', 'martini', 'anis', 'plastic', 'white wine', 'tennis ball', 'bbq', 'wet plaster', 'champagne', 'foresty', 'cognac'] 
Total number of unique notes: 80


In [20]:
# Initialize a dictionary to store unique fragrance notes and their frequencies.
note_counts = {} # Renamed 'unique_notes_dict' to 'note_counts'

# Iterate through each perfume's main accords to count the occurrences of each note.
# Each note is standardized to lowercase, and "unknown" notes are excluded from the count.
for accord_list in df_perfume_catalog.main_accords:
    for note in accord_list:
        note = note.lower()
        if note != "unknown":
            note_counts[note] = note_counts.get(note, 0) + 1 

In [21]:
# Create a DataFrame from the note counts for easier analysis and storage.
fragrance_note_frequency_df = pd.DataFrame(data={
    "Note": list(note_counts.keys()),
    "Count": list(note_counts.values())
}) 


## Women's perfumes sales dataset cleaning

In [23]:
# Load the eBay women's perfume dataset from a CSV file
ebay_womens_df = pd.read_csv("Datasets/ebay_womens_perfume.csv")

# Print the dimensions (number of rows and columns) of the DataFrame.
print(f"Shape of ebay_womens_df: {ebay_womens_df.shape}")

# Print the column names of the DataFrame.
print(f"Columns of ebay_womens_df:\n{ebay_womens_df.columns.tolist()}")

# Display a random sample of 5 rows from the DataFrame.
print("Sample of ebay_womens_df:")
display(ebay_womens_df.sample(5))

Shape of ebay_womens_df: (1000, 10)
Columns of ebay_womens_df:
['brand', 'title', 'type', 'price', 'priceWithCurrency', 'available', 'availableText', 'sold', 'lastUpdated', 'itemLocation']
Sample of ebay_womens_df:


,brand,title,type,price,priceWithCurrency,available,availableText,sold,lastUpdated,itemLocation
346,Oscar de la Renta,Oscar by Oscar De La Renta EDT Perfume for Wom...,Eau de Toilette,33.34,US $33.34/ea,8.0,"8 available / 3,466 sold",3466.0,"May 23, 2024 10:04:49 PDT","Hackensack, New Jersey, United States"
638,Britney Spears,CURIOUS Britney Spears women perfume edp 3.3 o...,Eau de Parfum,18.90,US $18.90/ea,245.0,"245 available / 3,860 sold",3860.0,"May 21, 2024 12:14:06 PDT","Dallas, Texas, United States"
885,Marc Jacobs,Daisy Marc Jacobs Eau So Fresh POP 2.5 oz/ 75 ...,Eau de Toilette,74.99,US $74.99/ea,10.0,10 available / 13 sold,13.0,"May 20, 2024 15:45:05 PDT","Katy, Texas, United States"
237,Narciso Rodriguez,Narciso Rodriguez for Her EDP 3.3oz - New Seal...,Eau de Toilette,34.99,US $34.99/ea,6.0,6 available / 5 sold,5.0,"May 23, 2024 10:31:01 PDT","Flat Lick, Kentucky, United States"
41,Kilian,Love by Kilian Don't Be Shy Eau de Parfum 7.5 ...,Eau de Parfum,29.75,US $29.75,9.0,9 available / 80 sold,80.0,"May 24, 2024 09:49:39 PDT","Tulsa, Oklahoma, United States"


In [24]:
# Drop columns that are not relevant for the analysis.
columns_to_drop = ["priceWithCurrency", "available", "availableText", "lastUpdated", "itemLocation"]
ebay_womens_df = ebay_womens_df.drop(columns=columns_to_drop)
ebay_womens_df.sample(5)

,brand,title,type,price,sold
98,Clinique,Clinique Happy by Clinique 3.3 / 3.4 oz Perfum...,Perfume,21.27,61.0
510,Coach,Coach Love 3 oz EDP Perfume for Women New In Box,Eau de Parfum,56.97,254.0
923,Montblanc,Montblanc - Signature Eau De Parfum Spray 50m...,Fragrances,31.01,6.0
144,Versace,VERSACE WOMAN by Gianni Versace Perfume for Wo...,Eau de Parfum,33.38,2715.0
522,Jo Malone,Jo Malone Wood Sage & Sea Salt 1 fl oz No Box ...,Eau de Cologne,34.99,12.0


In [25]:
# Analyze the contents and data types of the columns in the DataFrame.
print("DataFrame Information:")
print(ebay_womens_df.info())

# Check for missing values in each column of the DataFrame and print the sum of null values per column.
print("\nMissing Values per Column:")
print(ebay_womens_df.isnull().sum())


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   brand   999 non-null    object 
 1   title   1000 non-null   object 
 2   type    998 non-null    object 
 3   price   1000 non-null   float64
 4   sold    984 non-null    float64
dtypes: float64(2), object(3)
memory usage: 39.2+ KB
None

Missing Values per Column:
brand     1
title     0
type      2
price     0
sold     16
dtype: int64


In [26]:
# Inspect rows where the 'sold' column has missing values.
print("Rows with missing values in the 'sold' column:")
display(ebay_womens_df[ebay_womens_df.sold.isnull()])

# Remove rows where the 'sold' column has missing values.
ebay_womens_df = ebay_womens_df.dropna(subset=['sold'])

# Verify that there are no more missing values in the 'sold' column.
print("\nNumber of missing values in 'sold' after dropping rows:", ebay_womens_df.sold.isnull().sum())


Rows with missing values in the 'sold' column:


,brand,title,type,price,sold
4,Khadlaj,Shiyaaka for Men EDP Spray 100ML (3.4 FL.OZ) B...,Eau de Parfum,29.99,NaN
114,PRADA,Prada Paradoxe EDP Mini Bottle 7ml/.23oz,Eau de Parfum,19.00,NaN
117,Ouai,OUAI Hair & Body Mist In ST. BARTS 3.3oz / 97m...,Fragrance Mist,28.00,NaN
121,Ellis Brooklyn,NWB Ellis Brooklyn Miami Nectar Eau de Parfum ...,Eau de Parfum,33.00,NaN
432,Maison Alhambra,DELILAH PERFUME EDP 3.4 OZ BY MAISON ALHAMBRA ...,Eau de Parfum,23.99,NaN
496,Victoria's Secret,Victoria's Secret Bombshell Sundrenched Eau De...,Eau de Parfum,59.99,NaN
551,Jean couturier,CORIANDRE by JEAN COUTURIER for Women 30ml-1oz...,Eau de Toilette,21.95,NaN
602,Giardini Di Toscana,Bianco Latte EDP by Giardini Di Toscana 100 M...,Perfume,140.00,NaN
617,Dossier,Dossier MUSKY MUSK Eau de Parfum 1.7 Fl oz / 5...,Eau de Parfum,20.00,NaN
732,CHANEL,CHANEL Les Eaux De Chanel Paris Eau De Toilett...,Eau de Toilette,24.00,NaN



Number of missing values in 'sold' after dropping rows: 0


In [27]:
# Inspect rows where the 'type' column has missing values to determine if information can be extracted
#from the 'title' column.
print("Rows with missing values in the 'type' column:")
display(ebay_womens_df[ebay_womens_df.type.isna()])

# Manually fill the missing 'type' value for row 463 based on information from the 'title' column.
ebay_womens_df.at[463, 'type'] = "EDP"

# Verify the change by displaying the row 463.
print("\nRow 463 after filling the missing 'type' value:")
print(ebay_womens_df.loc[463])


Rows with missing values in the 'type' column:


,brand,title,type,price,sold
463,As Show,Carmina By Creed EDP 75ml 2.5 oz Spray For Wom...,NaN,128.88,21.0



Row 463 after filling the missing 'type' value:
brand                                              As Show
title    Carmina By Creed EDP 75ml 2.5 oz Spray For Wom...
type                                                   EDP
price                                               128.88
sold                                                  21.0
Name: 463, dtype: object


In [28]:
# Check for duplicate rows in the DataFrame.
print("Duplicate row counts:")
print(ebay_womens_df.duplicated().value_counts())

# Remove duplicate rows from the DataFrame.
ebay_womens_df = ebay_womens_df.drop_duplicates()

Duplicate row counts:
False    982
True       2
Name: count, dtype: int64


In [29]:
# Convert the data type of the 'sold' column (number of units sold) to integer.
ebay_womens_df.sold = ebay_womens_df.sold.astype(int)

In [30]:
# Check how many price entries contain a comma, indicating incorrect decimal formatting 
ebay_womens_df.price.astype(str).str.contains(",").sum()

0

In [31]:
# Standardize and analyze the 'type' column to consolidate categorical values and identify distribution
ebay_womens_df['type'] = ebay_womens_df.type.astype(str).str.lower().str.strip()
ebay_womens_df['type'].value_counts().to_dict()

{'eau de parfum': 569,
 'eau de toilette': 189,
 'perfume': 53,
 'eau de cologne': 38,
 'eau de perfume': 23,
 'fragrances': 14,
 'fragrance mist': 10,
 'eau de toilette, spray': 6,
 'spray': 4,
 'gift sets': 4,
 'lotion': 4,
 'perfume oil': 3,
 'cologne': 3,
 'extrait de parfum': 3,
 'parfum': 3,
 'eau de parfume': 2,
 'esprit de parfum': 2,
 "l'eau de parfum": 2,
 'does not apply': 2,
 'edp': 2,
 'perfume, eau de parfum': 2,
 'skin_moisturizer': 2,
 'perfume gift sets': 1,
 'eau de parfum spray (unisex tester) 3.4 oz': 1,
 'eau de parfum supreme': 1,
 'toilette spray': 1,
 'body powder': 1,
 'shimmering body oil': 1,
 'eau de toilette spray 3.4 oz': 1,
 'body lotion': 1,
 'deodorant': 1,
 'solid perfume stick': 1,
 'beauty': 1,
 'eau de parfum 3 pcs set': 1,
 'sensuous body moisturizer': 1,
 'cologne spray': 1,
 'pink sugar': 1,
 'oil perfume': 1,
 'fragrance body mist': 1,
 'eau de parfum/perfume': 1,
 'extract parfum': 1,
 'scented oils': 1,
 'asst': 1,
 'body mist': 1,
 '~ body fi

In [32]:
# Standardize 'perfume' spellings to 'parfum' for consistency.
ebay_womens_df.type = ebay_womens_df.type.str.replace('p[e|a]rfume', 'parfum', regex=True)

# Normalize and standardize the 'type' column using rule-based mappings and title content analysis.
# This ensures consistent categorization of fragrance types across the dataset.
ebay_womens_df['title'] = ebay_womens_df['title'].str.strip().str.lower() 

for index, row in ebay_womens_df.iterrows(): 
    fragrance_type = row["type"] 
    product_title = row["title"] 

    # Apply initial rule-based mapping for common type inconsistencies.
    if ("parfum" in fragrance_type and fragrance_type not in ["eau de parfum", "parfum"]) or fragrance_type in ["spray", "beauty", "edp", "1", "/"]:
        ebay_womens_df.at[index, "type"] = "eau de parfum"
    elif ("toilette" in fragrance_type and fragrance_type != "eau de toilette") or fragrance_type == "edt":
        ebay_womens_df.at[index, "type"] = "eau de toilette"
    elif "cologne" in fragrance_type and fragrance_type != "eau de cologne":
        ebay_womens_df.at[index, "type"] = "eau de cologne"
    elif fragrance_type in ["body spray", "mist"] or "mist" in fragrance_type:
        ebay_womens_df.at[index, "type"] = "mist"
    elif fragrance_type == "parfum":
        ebay_womens_df.at[index, "type"] = "fragrances"

    # Re-evaluate 'type' based on 'product_title' for entries still categorized as 'fragrances'.
    updated_fragrance_type = ebay_womens_df.at[index, "type"] 

    if updated_fragrance_type == "fragrances":
        if "edp" in  product_title or "eau de parfum" in  product_title or "parfum" in product_title:
            ebay_womens_df.at[index, "type"] = "eau de parfum"
        elif "edt" in  product_title or "eau de toilette" in  product_title:
            ebay_womens_df.at[index, "type"] = "eau de toilette"
        elif "cologne" in  product_title:
            ebay_womens_df.at[index, "type"] = "eau de cologne"
        elif "mist" in  product_title or "body mist" in  product_title:
            ebay_womens_df.at[index, "type"] = "mist"
        else:
            ebay_womens_df.at[index, "type"] = "other"

In [33]:
selected_fragrances_types = ["eau de parfum", "eau de toilette", "eau de cologne", "mist"]
# Filter the dataset to include only fragrance types relevant for the current analysis scope
ebay_womens_df = ebay_womens_df[ebay_womens_df['type'].isin(selected_fragrances_types)]

In [34]:
ebay_womens_df.type.value_counts()

type
eau de parfum      658
eau de toilette    201
eau de cologne      45
mist                17
Name: count, dtype: int64

In [35]:
# Normalize brand names by converting to lowercase and removing leading/trailing whitespace
ebay_womens_df.brand = ebay_womens_df.brand.str.lower().str.strip()

# Print the number of unique brands and the DataFrame's shape after initial normalization
print("n. unique brands:", ebay_womens_df.brand.nunique(), "on shape:", ebay_womens_df.shape)

n. unique brands: 209 on shape: (921, 5)


In [36]:
# Convert brand value counts to a dictionary for quick lookup or further analysis (optional).
#ebay_womens_df.brand.value_counts().to_dict()

In [37]:
# Identify and inspect brand entries containing "show" which often indicate generic descriptions like "as shown".
# These will be treated as missing values and attempted to be populated from the product title.
ebay_womens_df[ebay_womens_df.brand.str.contains("show")].brand.value_counts()

brand
as shown           17
as show            15
as  shown           1
as picture show     1
as showed           1
Name: count, dtype: int64

In [38]:
# Extract a list of known, non-generic brand names.
known_brands = ebay_womens_df.brand[ebay_womens_df.brand != 'as shown'].unique()

In [39]:
# Iterate through entries where the brand is 'as shown' and attempt to infer the correct brand from the product title.
# This helps to fill missing brand information using available textual data.
for idx, row in ebay_womens_df[ebay_womens_df['brand'] == 'as shown'].iterrows():
    for brand in known_brands: 
        if brand in row['title']:
            ebay_womens_df.at[idx, 'brand'] = brand
            break  # Stop at the first brand found in the title

In [40]:
# Display titles for records still marked with "show" in brand, to further understand remaining unassigned cases.
ebay_womens_df[ebay_womens_df.brand.str.contains("show")].title.to_dict()

{3: "j'adore parfum d'eau by christian 3.4 oz edp fragrance for women new in box",
 6: 'prada paradoxe by prada edp 3.0oz/90ml spray perfume for women new in box us',
 35: 'perfume edp for women 3.4 oz / 100 ml brand new',
 48: 'flora gorgeous jasmine 3.3 oz / 100 ml edp perfume spray for women new in box',
 92: 'women perfume 3.4 oz / 100 ml eau de toilette spray',
 124: "j'adore 3.4 oz/100 ml eau de parfum edp women spray gift for her new & sealed us",
 134: 'flora gorgeous magnolia by-gucci eau de parfum edp 3.3 oz perfume for women nib',
 142: '3.3 oz 100 ml flora gorgeous gardena eau de parfum edp for women new in box',
 189: 'prada paradoxe by prada edp 3.0oz/90ml spray perfume for women new in box',
 208: 'black orchid by tom ford3.4 oz / 100 ml edp for women new in box regular size',
 384: 'woman edt spray perfume 3.4oz new',
 393: 'addict by christian edp fragrances for women 3.4 oz / 100 ml new in sealed box',
 448: 'new with box for her eau de toilette narciso_rodriguez edt 

In [41]:
# Standardize remaining 'as shown' variations to a consistent 'as shown' value.
for index in ebay_womens_df[ebay_womens_df.brand.str.contains("show")].index:
    ebay_womens_df.at[index, "brand"] = 'as shown'

In [42]:
# Correct specific brand misspellings or variations to their standardized form.
# This loop specifically addresses '-al rehab-' variations.
for index in ebay_womens_df[ebay_womens_df.brand.str.contains("-")].index:
    ebay_womens_df.at[index, "brand"] = "al rehab"

# Inspect brand names starting with 'y' to identify other potential variations (e.g., 'ysl').
ebay_womens_df[ebay_womens_df.brand.str.startswith('y')].brand.value_counts()
# Standardize 'ysl' to 'yves saint laurent'.
for index in ebay_womens_df[ebay_womens_df.brand == "ysl"].index:
    ebay_womens_df.at[index, "brand"] = "yves saint laurent"

# Standardize 'tf' to 'tom ford'.
for index in ebay_womens_df[ebay_womens_df.brand == "tf"].index:
    ebay_womens_df.at[index, "brand"] = "tom ford"

# Standardize 'maison martin margiela' to 'maison margiela'.
for index in ebay_womens_df[ebay_womens_df.brand == "maison martin margiela"].index:
    ebay_womens_df.at[index, "brand"] = "maison margiela"

# Standardize 'lattafa ' variations to 'lattafa'.
for index in ebay_womens_df[ebay_womens_df.brand.str.contains("lattafa ")].index:
    ebay_womens_df.at[index, "brand"] = "lattafa"
    
# Standardize 'huda beauty kayali' to 'huda beauty' for consistent branding.
for index in ebay_womens_df[ebay_womens_df.brand == "huda beauty kayali"].index:
    ebay_womens_df.at[index, "brand"] = "huda beauty"

# Correct 'chloe' to 'chloé' to reflect the accurate brand spelling.
for index in ebay_womens_df[ebay_womens_df.brand == "chloe"].index:
    ebay_womens_df.at[index, "brand"] = "chloé"

# Expand 'benetton' to its full official name, 'united colors of benetton'.
for index in ebay_womens_df[ebay_womens_df.brand == "benetton"].index:
    ebay_womens_df.at[index, "brand"] = "united colors of benetton"

# Rectify 'lancome' to 'lancôme' for correct diacritical mark usage.
for index in ebay_womens_df[ebay_womens_df.brand == "lancome"].index:
    ebay_womens_df.at[index, "brand"] = "lancôme"

# Consolidate 'dolce ' variations to 'dolce&gabbana' for a unified brand entry.
for index in ebay_womens_df[ebay_womens_df.brand.str.contains("dolce ")].index:
    ebay_womens_df.at[index, "brand"] = "dolce&gabbana"

# Normalize various 'giorgio armani' misspellings and formats to a standard 'giorgio armani'.
for index in ebay_womens_df[ebay_womens_df.brand.isin(['giorgio² armani', 'giorgio arm.ani', 'giorgi^o armani'])].index:
    ebay_womens_df.at[index, "brand"] = "giorgio armani"

## Men's parfumes sales dataset cleaning 


In [44]:
# Load the eBay men's perfume dataset from the specified CSV file.
ebay_mens_df = pd.read_csv("Datasets/ebay_mens_perfume.csv")

# Display the dimensions (rows, columns) of the DataFrame and its column names for initial assessment.
print(f"DataFrame shape: {ebay_mens_df.shape}")
print(f"DataFrame columns: {ebay_mens_df.columns.tolist()}") # Use .tolist() for better readability

# Sample 5 random rows to get a quick overview of the data structure and content.
ebay_mens_df.sample(5)

DataFrame shape: (1000, 10)
DataFrame columns: ['brand', 'title', 'type', 'price', 'priceWithCurrency', 'available', 'availableText', 'sold', 'lastUpdated', 'itemLocation']


,brand,title,type,price,priceWithCurrency,available,availableText,sold,lastUpdated,itemLocation
441,Lacoste,Lacoste Essential Eau De Toilette Pour Homme ~...,Eau de Toilette,31.95,US $31.95,10.0,More than 10 available / 12 sold,12.0,"May 15, 2024 07:37:26 PDT","Oneida, Tennessee, United States"
239,Unbranded,1/2PCS 50ml Men's Pheromone-Cupid Infused Perf...,Perfume,12.98,US $12.98,10.0,10 available / 103 sold,103.0,"May 24, 2024 10:10:30 PDT","Perth Amboy, New Jersey, United States"
80,Unbranded,SEALED NEW CUPID HYPNOSIS MEN’S PHEROMONE COLO...,Perfume,15.99,US $15.99,10.0,More than 10 available / 594 sold,594.0,"May 24, 2024 09:12:14 PDT","Houston, Texas, United States"
97,Cologne,BLEU PARFUM de Blue for Men 3.4oz / 100ml EAU ...,Perfume,85.00,US $85.00,4.0,4 available / 42 sold,42.0,"May 23, 2024 22:52:13 PDT","Fresh Meadows, New York, United States"
770,PRADA,Prada Luna Rossa Black EDP Eau De Parfum 8ml S...,Eau de Parfum,22.99,US $22.99,9.0,9 available / 1 sold,1.0,NaN,"Cincinnati, Ohio, United States"


In [45]:
# Drop columns irrelevant to the current analysis to streamline the dataset.
columns_to_drop = ["priceWithCurrency", "available", "availableText", "lastUpdated", "itemLocation"]
ebay_mens_df = ebay_mens_df.drop(columns=columns_to_drop)

In [46]:
# Display 5 random rows again to confirm the successful removal of columns.
ebay_mens_df.sample(5)

,brand,title,type,price,sold
336,Givenchy,Givenchy Gentleman 3.3oz.Eau de Toilette INTEN...,Eau de Toilette Intense,59.00,601.0
996,Armaf,Club de Nuit Intense by Armaf cologne for men ...,Eau de Toilette,30.58,31.0
348,Montblanc,Legend Red by Mont Blanc cologne for men EDP 3...,Eau de Parfum,31.29,349.0
604,Paco Rabanne,Invictus by Paco Rabanne cologne for men EDT 6...,Eau de Toilette,77.15,293.0
684,Yves Saint Laurent,NIB YSL Yves Saint Laurent MYSLF EDP Spray 3ml...,Eau de Parfum,15.99,91.0


In [47]:
# Display concise summary of the DataFrame, including data types and non-null values.
print(ebay_mens_df.info())
# Count and display the number of null values for each column.
print("\nNull values before handling:\n",ebay_mens_df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   brand   999 non-null    object 
 1   title   1000 non-null   object 
 2   type    997 non-null    object 
 3   price   1000 non-null   float64
 4   sold    994 non-null    float64
dtypes: float64(2), object(3)
memory usage: 39.2+ KB
None

Null values before handling:
 brand    1
title    0
type     3
price    0
sold     6
dtype: int64


In [48]:
# Display rows where 'brand' is null to understand specific cases needing manual intervention.
display(ebay_mens_df[ebay_mens_df.brand.isnull()])
# Manually assign 'Versace' as the brand for a specific missing entry (row 567).
ebay_mens_df.at[567, "brand"] = "Versace"

,brand,title,type,price,sold
567,NaN,Versace Pour Homme Oud Noir 3.4 oz EDP Cologne...,Eau de Parfum,55.16,3110.0


In [49]:
# Display rows where 'type' is null for targeted manual correction.
display(ebay_mens_df[ebay_mens_df.type.isnull()])
# Manually assign fragrance types for specific missing entries based on external knowledge or inspection.
ebay_mens_df.at[81, "type"] = "Cologne"
ebay_mens_df.at[138, "type"] = "EDP"
ebay_mens_df.at[606, "type"] = "EDP"

,brand,title,type,price,sold
81,Calvin Klein,Ck One by Calvin Klein Cologne Perfume Unisex ...,NaN,23.89,54052.0
138,AS SHOW,Bleu De Eau de parfum EDP 100ml 3.4 oz Cologne...,NaN,89.99,5.0
606,Milestone Perfumes,Intimation By Milestone Eau de Parfum 3.4 oz Men,NaN,19.89,21.0


In [50]:
# Display rows where 'sold' (number of units sold) is missing.
display(ebay_mens_df[ebay_mens_df.sold.isna()])
# Remove rows where 'sold' data is missing, as this is a critical metric for analysis.
ebay_mens_df = ebay_mens_df.dropna(subset=['sold'])
# Convert the 'sold' column to integer type for numerical operations.
ebay_mens_df.sold = ebay_mens_df.sold.astype(int)

,brand,title,type,price,sold
152,Dossier,Dossier Aromatic Pineapple Eau de Parfum. Size...,Eau de Parfum,20.00,NaN
405,Jean Paul Gaultier,Jean Paul Gaultier Le Beau Le Parfum Eau De Pa...,Eau de Parfum,169.00,NaN
686,CHANEL,CHANEL Allure Homme Sport 3.4 Oz Men Eau De To...,Eau de Toilette,89.97,NaN
792,Bath & Body Works,Bath & Body Works Men's Collection FRESHWATER ...,Cologne,29.97,NaN
797,Tom Ford,"5 Original M&H Tom Ford Impressions, Tobacco V...",Fragrance Oil,29.99,NaN
899,Bvlgari,Bvlgari Le Gemme Tygar 3.4oz Men's Eau de Parfum,Eau de Parfum,250.00,NaN


In [51]:
# Check for duplicate rows in the DataFrame.
print(f"\nNumber of duplicate rows: {ebay_mens_df.duplicated().sum()}")

# Verify if any 'price' entries contain commas, which would indicate a need for string-to-numeric conversion.
print(f"Number of prices with commas: {ebay_mens_df.price.astype(str).str.contains(',').sum()}")


Number of duplicate rows: 0
Number of prices with commas: 0


In [52]:
# Standardize 'type' column: convert to lowercase and remove leading/trailing whitespace.
ebay_mens_df.type = ebay_mens_df.type.astype(str).str.lower().str.strip()
# Display value counts of 'type' as a dictionary for an overview of current categories.
ebay_mens_df.type.value_counts().to_dict()

{'eau de toilette': 453,
 'eau de parfum': 297,
 'eau de cologne': 57,
 'perfume': 50,
 'fragrances': 32,
 'parfum': 17,
 'eau de perfume': 10,
 'gift sets': 6,
 'cologne': 6,
 'body spray': 4,
 'edt': 4,
 'aftershave': 3,
 'does not apply': 3,
 'parfum intense': 3,
 'eau de parfum intense': 3,
 'extrait de parfum': 3,
 'le parfum': 3,
 '/': 3,
 'pheromone': 2,
 'edp': 2,
 'fragrance rolling ball': 2,
 'elixir': 1,
 'elixir de parfum': 1,
 'roll on': 1,
 'various': 1,
 'assorted': 1,
 'edc': 1,
 'eau de parfum/ eau de toilette': 1,
 'deodorant body spray': 1,
 'oil': 1,
 'splash-on': 1,
 'car air freshener': 1,
 'fragrance body spray': 1,
 'body oil': 1,
 'eau de cologne spray': 1,
 'eau de toilette intense': 1,
 'eau de toilette, cologne spray': 1,
 'de nuit': 1,
 'deodorant': 1,
 'dior homme cologne': 1,
 'editions parfums': 1,
 'fine cologne': 1,
 'eau de cologne spray, cologne spray': 1,
 'eau de toilette, eau de parfum, eau de parfum intense': 1,
 'eau de toillette': 1,
 'y': 1,
 

In [53]:
# Normalize inconsistent spellings of 'perfume' to 'parfum' in the 'type' column.
ebay_mens_df.type = ebay_mens_df.type.str.replace('p[e|a]rfume', 'parfum', regex=True)

# Clean and standardize 'title' column: convert to lowercase and remove leading/trailing whitespace.
ebay_mens_df.title = ebay_mens_df.title.str.strip().str.lower()

In [54]:
# Iterate through each row to normalize and standardize the 'type' column
# based on predefined rules and content from the 'title' column.
for i in ebay_mens_df.index:
    current_type = ebay_mens_df.at[i, "type"] # Renamed 'tipo' to 'current_type' for clarity
    product_title = ebay_mens_df.at[i, "title"] # Renamed 'title' to 'product_title' for clarity

    # Apply initial rule-based mapping for common type inconsistencies.
    if ("parfum" in current_type and current_type not in ["eau de parfum", "parfum"]) or current_type in ["edp", "/"]:
        ebay_mens_df.at[i, "type"] = "eau de parfum"
    elif "toilette" in current_type or current_type == "edt" or "toillette" in current_type:
        ebay_mens_df.at[i, "type"] = "eau de toilette"
    elif "cologne" in current_type or current_type == "edc":
        ebay_mens_df.at[i, "type"] = "eau de cologne"
    elif current_type == "parfum":
        ebay_mens_df.at[i, "type"] = "fragrances"

    # Re-evaluate 'type' based on 'product_title' if the type is still ambiguous.
    updated_type = ebay_mens_df.at[i, "type"] 

    if updated_type not in ["eau de cologne", "eau de toilette", "eau de parfum"]:
        if "edp" in product_title or "eau de parfum" in product_title or "parfum" in product_title:
            ebay_mens_df.at[i, "type"] = "eau de parfum"
        elif "edt" in product_title or "eau de toilette" in product_title:
            ebay_mens_df.at[i, "type"] = "eau de toilette"
        elif "cologne" in product_title:
            ebay_mens_df.at[i, "type"] = "eau de cologne"
        elif "spray" in product_title or "pheromone" in product_title:
            ebay_mens_df.at[i, "type"] = "body spray"
        else:
            ebay_mens_df.at[i, "type"] = "other"

In [55]:
# Display the value counts of the 'type' column after standardization.
ebay_mens_df.type.value_counts()

type
eau de toilette    471
eau de parfum      392
eau de cologne     109
other               16
body spray           6
Name: count, dtype: int64

In [56]:
# Define the list of primary fragrance categories for men's perfumes.
men_fragrance_categories = ["eau de parfum", "eau de toilette", "eau de cologne", "body spray"] 
# Filter the DataFrame to include only the defined relevant fragrance categories.
ebay_mens_df = ebay_mens_df[ebay_mens_df.type.isin(men_fragrance_categories)]
# Display the value counts of 'type' after final filtering.
ebay_mens_df.type.value_counts()

type
eau de toilette    471
eau de parfum      392
eau de cologne     109
body spray           6
Name: count, dtype: int64

In [57]:
# Normalize brand names by converting to lowercase and removing leading/trailing whitespace.
ebay_mens_df.brand = ebay_mens_df.brand.str.lower().str.strip()

# Inspect brand names starting with 'a' for specific normalization needs (e.g., 'as shown', 'acqua di gio').
ebay_mens_df[ebay_mens_df.brand.str.startswith('a')].brand.value_counts()

# Standardize 'ysl' to 'yves saint laurent'.
for index in ebay_mens_df[ebay_mens_df.brand == 'ysl'].index:
    ebay_mens_df.at[index, "brand"] = "yves saint laurent"

# Correct 'victor & rolf' to 'viktor & rolf' for accurate spelling.
for index in ebay_mens_df[ebay_mens_df.brand == 'victor & rolf'].index:
    ebay_mens_df.at[index, "brand"] = "viktor & rolf"

# Consolidate 'roja parfums' to 'roja' for brevity and consistency.
for index in ebay_mens_df[ebay_mens_df.brand == 'roja parfums'].index:
    ebay_mens_df.at[index, "brand"] = "roja"

# Standardize 'mont blanc' to 'montblanc' (single word).
for index in ebay_mens_df[ebay_mens_df.brand == 'mont blanc'].index:
    ebay_mens_df.at[index, "brand"] = "montblanc"

# Normalize 'lauren ralph lauren' to 'ralph lauren' for simplified branding.
for index in ebay_mens_df[ebay_mens_df.brand == 'lauren ralph lauren'].index:
    ebay_mens_df.at[index, "brand"] = "ralph lauren"
    
# Apply the correct diacritical mark to 'hermes', changing it to 'hermès'.
for index in ebay_mens_df[ebay_mens_df.brand == 'hermes'].index:
    ebay_mens_df.at[index, "brand"] = "hermès"
    
# Standardize 'guerlain paris' to 'guerlain' for conciseness and consistency.
for index in ebay_mens_df[ebay_mens_df.brand == 'guerlain paris'].index:
    ebay_mens_df.at[index, "brand"] = "guerlain"

# Standardize 'gianni versace' to the more common and concise 'versace'.
for index in ebay_mens_df[ebay_mens_df.brand == 'gianni versace'].index:
    ebay_mens_df.at[index, "brand"] = "versace"

# Correct the misspelling 'giorgio arm.ani' to the accurate 'giorgio armani'.
for index in ebay_mens_df[ebay_mens_df.brand == 'giorgio arm.ani'].index:
    ebay_mens_df.at[index, "brand"] = "giorgio armani"

# Apply the correct diacritical mark to 'estee lauder', changing it to 'estée lauder'.
for index in ebay_mens_df[ebay_mens_df.brand == 'estee lauder'].index:
    ebay_mens_df.at[index, "brand"] = "estée lauder"

# Consolidate various 'dolce &' formats to the standardized 'dolce&gabbana'.
for index in ebay_mens_df[ebay_mens_df.brand.str.contains("dolce &")].index:
    ebay_mens_df.at[index, "brand"] = "dolce&gabbana"

# Normalize all 'dior' variations to a consistent 'dior'.
for index in ebay_mens_df[ebay_mens_df.brand.str.contains("dior")].index:
    ebay_mens_df.at[index, "brand"] = "dior"

# Expand the abbreviation 'c.k' to its full brand name, 'calvin klein'.
for index in ebay_mens_df[ebay_mens_df.brand == 'c.k'].index:
    ebay_mens_df.at[index, "brand"] = "calvin klein"

In [58]:
# Identify and extract all unique, known brand names from the dataset, excluding generic placeholders like 'as shown'.
known_brands = ebay_mens_df.brand[ebay_mens_df.brand != 'as shown'].unique()

# Attempt to impute missing or generic 'as shown' brand values by searching for known brands within the product 'title'.
# This loop iterates through entries with 'as shown' as their brand and updates the brand if a known brand is found in the title.
for idx, row in ebay_mens_df[ebay_mens_df['brand'] == 'as shown'].iterrows():
    for brand_name in known_brands:
        if brand_name in row['title']:
            ebay_mens_df.at[idx, 'brand'] = brand_name
            break  # Stop at the first matching brand found to avoid multiple assignments.

# Exporting cleaned DataFrames

In [60]:
# Export the cleaned main perfume catalog DataFrame to a CSV file.
df_perfume_catalog.to_csv('parfumes_df_cleaned.csv', index=False)

# Export the DataFrame containing fragrance note frequencies to a CSV file.
fragrance_note_frequency_df.to_csv('note_frequencies.csv', index=False)

# Export the cleaned men's perfume DataFrame to a CSV file.
ebay_mens_df.to_csv('ebay_mens_df_cleaned.csv', index=False)

# Export the cleaned women's perfume DataFrame to a CSV file.
ebay_womens_df.to_csv('ebay_womens_df_cleaned.csv', index=False)